# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 8 2023 9:33AM,258759,15,30014535,"Alliance Pharma, Inc.",Released
1,Mar 8 2023 9:33AM,258759,15,30014533,"Alliance Pharma, Inc.",Released
2,Mar 8 2023 9:33AM,258759,15,30014536,"Alliance Pharma, Inc.",Released
3,Mar 8 2023 9:33AM,258759,15,30014534,"Alliance Pharma, Inc.",Released
4,Mar 8 2023 9:33AM,258759,15,30014538,"Alliance Pharma, Inc.",Released
5,Mar 8 2023 9:33AM,258759,15,30014537,"Alliance Pharma, Inc.",Released
6,Mar 8 2023 9:33AM,258759,15,30014563,"Alliance Pharma, Inc.",Released
7,Mar 8 2023 9:33AM,258759,15,30014557,"Alliance Pharma, Inc.",Released
8,Mar 8 2023 9:33AM,258759,15,30014558,"Alliance Pharma, Inc.",Released
9,Mar 8 2023 9:33AM,258759,15,30014559,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,258928,Released,1
30,258929,Released,12
31,258930,Released,1
32,258931,Released,1
33,258932,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258928,NaN,NaN,1.0
258929,NaN,NaN,12.0
258930,NaN,NaN,1.0
258931,NaN,NaN,1.0
258932,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258712,0.0,0.0,37.0
258759,0.0,0.0,31.0
258811,12.0,1.0,2.0
258816,0.0,1.0,0.0
258828,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258712,0,0,37
258759,0,0,31
258811,12,1,2
258816,0,1,0
258828,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258712,0,0,37
1,258759,0,0,31
2,258811,12,1,2
3,258816,0,1,0
4,258828,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258712,,,37
1,258759,,,31
2,258811,12,1,2
3,258816,,1,
4,258828,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc."
31,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc."
68,Mar 8 2023 9:31AM,258932,10,"Citieffe, Inc."
69,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc."
70,Mar 8 2023 9:23AM,258929,10,"Methapharm, Inc."
82,Mar 8 2023 9:23AM,258930,10,"DermaCare BioSciences, LTD."
83,Mar 8 2023 9:16AM,258928,10,Eye Pharma Inc
84,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC"
88,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc."
96,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",,,31
1,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",,,37
2,Mar 8 2023 9:31AM,258932,10,"Citieffe, Inc.",,,1
3,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",,,1
4,Mar 8 2023 9:23AM,258929,10,"Methapharm, Inc.",,,12
5,Mar 8 2023 9:23AM,258930,10,"DermaCare BioSciences, LTD.",,,1
6,Mar 8 2023 9:16AM,258928,10,Eye Pharma Inc,,,1
7,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",,,4
8,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",,,8
9,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",31,,
1,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",37,,
2,Mar 8 2023 9:31AM,258932,10,"Citieffe, Inc.",1,,
3,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",1,,
4,Mar 8 2023 9:23AM,258929,10,"Methapharm, Inc.",12,,
5,Mar 8 2023 9:23AM,258930,10,"DermaCare BioSciences, LTD.",1,,
6,Mar 8 2023 9:16AM,258928,10,Eye Pharma Inc,1,,
7,Mar 8 2023 9:10AM,258926,10,"Digital Brands, LLC",4,,
8,Mar 8 2023 8:57AM,258925,10,"Methapharm, Inc.",8,,
9,Mar 8 2023 8:52AM,258923,12,"SD Head USA, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",31,,
1,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",37,,
2,Mar 8 2023 9:31AM,258932,10,"Citieffe, Inc.",1,,
3,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",1,,
4,Mar 8 2023 9:23AM,258929,10,"Methapharm, Inc.",12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",31.0,NaN,NaN
1,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",37.0,NaN,NaN
2,Mar 8 2023 9:31AM,258932,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",1.0,NaN,NaN
4,Mar 8 2023 9:23AM,258929,10,"Methapharm, Inc.",12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",31.0,0.0,0.0
1,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",37.0,0.0,0.0
2,Mar 8 2023 9:31AM,258932,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",1.0,0.0,0.0
4,Mar 8 2023 9:23AM,258929,10,"Methapharm, Inc.",12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3106937,40.0,3.0,12.0
12,2330085,9.0,3.0,0.0
15,517471,68.0,0.0,0.0
19,1035564,8.0,19.0,0.0
21,517809,2.0,0.0,0.0
22,258912,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3106937,40.0,3.0,12.0
1,12,2330085,9.0,3.0,0.0
2,15,517471,68.0,0.0,0.0
3,19,1035564,8.0,19.0,0.0
4,21,517809,2.0,0.0,0.0
5,22,258912,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40.0,3.0,12.0
1,12,9.0,3.0,0.0
2,15,68.0,0.0,0.0
3,19,8.0,19.0,0.0
4,21,2.0,0.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,12,Released,9.0
2,15,Released,68.0
3,19,Released,8.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,12.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,3.0,0.0,19.0,0.0,1.0
Released,40.0,9.0,68.0,8.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,3.0,0.0,19.0,0.0,1.0
2,Released,40.0,9.0,68.0,8.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,3.0,0.0,19.0,0.0,1.0
2,Released,40.0,9.0,68.0,8.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()